## Code Document

Importing libraries and setting up authentication with Spotify API.

In [1]:
#Importing data from Spotify dataset

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Replace 'your_client_id', 'your_client_secret', and 'your_redirect_uri' with your actual values
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='5093ad0ad45f435ca7ce43b2afd406b9',
    client_secret='f151962f414144aa85494be81ccd8e20',
    redirect_uri='http://localhost:8888/callback',
    scope="user-library-read playlist-read-private"))

Fetching data from Spotify

In [ ]:
'''playlist_id = '6JOysTE0drK9yDi1Xs4FKy'  # Replace with the actual playlist ID
playlist = sp.playlist(playlist_id)
print(playlist['name'])
for item in playlist['tracks']['items']:
    track = item['track']
    print(track['name'], '-', track['artists'][0]['name'])'''

In [ ]:
import pandas as pd

# Fetch saved tracks
playlist_id = '6JOysTE0drK9yDi1Xs4FKy'

results = sp.playlist_tracks(playlist_id)
tracks_data = {
    'Name': [],
    'Artist': [],
    'Album': [],
    'Release Date': []
}

for item in results['items']:
    track = item.get('track')
    if track:  # Check if track details are present
        name = track.get('name', 'No Title Available')
        artist_name = track['artists'][0].get('name', 'No Artist Available') if track['artists'] else 'No Artists'
        album_name = track['album'].get('name', 'No Album Available') if track['album'] else 'No Album'
        release_date = track['album'].get('release_date', 'No Release Date Available') if track['album'] else 'No Release Info'
        
        tracks_data['Name'].append(name)
        tracks_data['Artist'].append(artist_name)
        tracks_data['Album'].append(album_name)
        tracks_data['Release Date'].append(release_date)

# Convert to DataFrame
df_tracks = pd.DataFrame(tracks_data)
print(df_tracks)

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Initialize the Spotify client
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id='your_client_id',
    client_secret='your_client_secret',
    redirect_uri='http://localhost:8888/callback',
    scope='user-library-read'))

# Search for tracks by genre
genre = "rock"  # Example genre
results = sp.search(q='genre:' + genre, type='track', limit=50)
tracks_data = {
    'Name': [],
    'Artist': [],
    'Genre': genre,
    'Popularity': []
}

for track in results['tracks']['items']:
    tracks_data['Name'].append(track['name'])
    tracks_data['Artist'].append(track['artists'][0]['name'])
    tracks_data['Popularity'].append(track['popularity'])

# Print or process the results
print([tracks_data])

In [ ]:
# Getting the playlist ID for Top 50 - Global playlist
playlist_id = '37i9dQZEVXbMDoHDwVN2tF'  # This is an example; use the current playlist ID for global top tracks

# Fetch tracks from the playlist
results = sp.playlist_tracks(playlist_id)
tracks_data = []
for item in results['items']:
    track = item['track']
    track_info = {
        'Name': track['name'],
        'Artist': track['artists'][0]['name'] if track['artists'] else 'Unknown',
        'Album': track['album']['name'] if track['album'] else 'Unknown',
        'Release Date': track['album']['release_date'] if track['album'] else 'Unknown',
        'Popularity': track['popularity']
    }
    tracks_data.append(track_info)

# Print or process the tracks data
for track in tracks_data:
    print(track)

#### Importing Song Data with Lyrics using API

In [2]:
import lyricsgenius

# Initialize Genius API
genius = lyricsgenius.Genius('put_your_token_here')

genius.remove_section_headers = True

# Spotify Global Top 50 playlist ID
playlist_id = '37i9dQZEVXbMDoHDwVN2tF'  # Example ID, update as necessary

results = sp.playlist_tracks(playlist_id)
tracks_data = []

# Fetch track details from Spotify
for item in results['items']:
    track = item['track']
    track_info = {
        'Name': track['name'],
        'Artist': track['artists'][0]['name'] if track['artists'] else 'Unknown',
        'Album': track['album']['name'] if track['album'] else 'Unknown',
        'Release Date': track['album']['release_date'] if track['album'] else 'Unknown',
        'Popularity': track['popularity']
    }
    # Fetch lyrics using Genius
    song = genius.search_song(track_info['Name'], track_info['Artist'])
    track_info['Lyrics'] = song.lyrics if song else 'Lyrics not found'
    
    tracks_data.append(track_info)

Searching for "Fortnight (feat. Post Malone)" by Taylor Swift...
Done.
Searching for "i like the way you kiss me" by Artemas...
Done.
Searching for "Espresso" by Sabrina Carpenter...
Done.
Searching for "Gata Only" by FloyyMenor...
Done.
Searching for "I Can Do It With a Broken Heart" by Taylor Swift...
Done.
Searching for "Down Bad" by Taylor Swift...
Done.
Searching for "Too Sweet" by Hozier...
Done.
Searching for "So Long, London" by Taylor Swift...
Done.
Searching for "Beautiful Things" by Benson Boone...
Done.
Searching for "End of Beginning" by Djo...
Done.
Searching for "My Boy Only Breaks His Favorite Toys" by Taylor Swift...
Done.
Searching for "we can't be friends (wait for your love)" by Ariana Grande...
Done.
Searching for "The Tortured Poets Department" by Taylor Swift...
Done.
Searching for "Who’s Afraid of Little Old Me?" by Taylor Swift...
Done.
Searching for "Florida!!! (feat. Florence + The Machine)" by Taylor Swift...
Done.
Searching for "But Daddy I Love Him" by Tay

In [3]:
import pandas as pd

# Convert the data to a pandas DataFrame
df_tracks = pd.DataFrame(tracks_data)

# Set column names and display the DataFrame
df_tracks.columns = ['Track Name', 'Artist', 'Album', 'Release Date', 'Popularity', 'Lyrics']
print(df_tracks)

                                           Track Name             Artist   
0                       Fortnight (feat. Post Malone)       Taylor Swift  \
1                          i like the way you kiss me            Artemas   
2                                            Espresso  Sabrina Carpenter   
3                                           Gata Only         FloyyMenor   
4                     I Can Do It With a Broken Heart       Taylor Swift   
5                                            Down Bad       Taylor Swift   
6                                           Too Sweet             Hozier   
7                                     So Long, London       Taylor Swift   
8                                    Beautiful Things       Benson Boone   
9                                    End of Beginning                Djo   
10               My Boy Only Breaks His Favorite Toys       Taylor Swift   
11           we can't be friends (wait for your love)      Ariana Grande   
12          

In [4]:
# Save DataFrame to CSV file
df_tracks.to_csv('spotify_songs_with_lyrics.csv', index=False)

In [7]:
df_tracks.columns

Index(['Track Name', 'Artist', 'Album', 'Release Date', 'Popularity',
       'Lyrics'],
      dtype='object')

### Preprocessing data

Clean the lyrics by removing punctuation, converting to lowercase, and other typical text cleaning steps:

In [8]:
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Removeing punctuation
    text = text.lower()  # Convert to lower case
    return text

# Apply the cleaning function to the lyrics column
df_tracks['cleaned_lyrics'] = df_tracks['Lyrics'].apply(clean_text)

### Vectorization

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_tracks['cleaned_lyrics'])

### Sentiment Analysis using VADER

In [13]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment intensity analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get the compound sentiment score
def get_sentiment(text):
    score = analyzer.polarity_scores(text)
    return score['compound']  # Return the compound score

# Apply the sentiment analysis function to the cleaned lyrics
df_tracks['sentiment_score'] = df_tracks['cleaned_lyrics'].apply(get_sentiment)

### Assign Sentiment Labels

Classifying the sentiments as positive, neutral or negative based on the computed score

In [14]:
def assign_sentiment(score):
    if score > 0.05:
        return 'positive'
    elif score < -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment label assignment
df_tracks['sentiment_label'] = df_tracks['sentiment_score'].apply(assign_sentiment)

In [17]:
print(df_tracks[['Track Name', 'Artist', 'sentiment_score', 'sentiment_label']])

                                           Track Name             Artist   
0                       Fortnight (feat. Post Malone)       Taylor Swift  \
1                          i like the way you kiss me            Artemas   
2                                            Espresso  Sabrina Carpenter   
3                                           Gata Only         FloyyMenor   
4                     I Can Do It With a Broken Heart       Taylor Swift   
5                                            Down Bad       Taylor Swift   
6                                           Too Sweet             Hozier   
7                                     So Long, London       Taylor Swift   
8                                    Beautiful Things       Benson Boone   
9                                    End of Beginning                Djo   
10               My Boy Only Breaks His Favorite Toys       Taylor Swift   
11           we can't be friends (wait for your love)      Ariana Grande   
12          

In [19]:
df_tracks[['Track Name', 'Artist', 'sentiment_score', 'sentiment_label']].to_csv('test_sentiment.csv', index=False)

test_sentiment = pd.read_csv('test_sentiment.csv')

#### Importing Json file to Pandas readable format

Start by importing spotify challenge (AI Crowd) file into python.

In [22]:
import pandas as pd
import json

# Load JSON data from file
with open('/Users/rishabhhasija/Documents/neuefische/sentify_recommender/challenge_set.json', 'r') as file:
    data = json.load(file)

# Initialize an empty list to collect all track data
all_tracks = []

# Loop through each playlist in the dataset
for playlist in data['playlists']:
    for track in playlist['tracks']:
        # Add playlist-level information to each track record
        track_info = {
            'playlist_name': playlist.get('name', 'Unknown'),
            'playlist_pid': playlist['pid'],
            'playlist_num_tracks': playlist['num_tracks'],
            'track_pos': track['pos'],
            'artist_name': track['artist_name'],
            'track_uri': track['track_uri'],
            'artist_uri': track['artist_uri'],
            'track_name': track['track_name'],
            'album_uri': track['album_uri'],
            'duration_ms': track['duration_ms'],
            'album_name': track['album_name']
        }
        all_tracks.append(track_info)

# Convert the list of track dictionaries to a DataFrame
df_spotify = pd.DataFrame(all_tracks)

# Display the first few rows of the DataFrame to verify
print(df_spotify.head())


  playlist_name  playlist_pid  playlist_num_tracks  track_pos artist_name   
0         Party       1000000                   75          0   AronChupa  \
1         Party       1000000                   75          1   AronChupa   
2         Party       1000000                   75          2       Lorde   
3         Party       1000000                   75          3       Lorde   
4         Party       1000000                   75          4       Lorde   

                              track_uri   
0  spotify:track:66U0ASk1VHZsqIkpMjKX3B  \
1  spotify:track:5MhsZlmKJG6X5kTHkdwC4B   
2  spotify:track:0GZoB8h0kqXn7XFm4Sj06k   
3  spotify:track:35kahykNu00FPysz3C2euR   
4  spotify:track:3G6hD9B2ZHOsgf4WfNu7X1   

                              artist_uri   
0  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh  \
1  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh   
2  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   
3  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   
4  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   

             

In [21]:
print(df_spotify.shape)
print(df_spotify.head())

(281000, 11)
  playlist_name  playlist_pid  playlist_num_tracks  track_pos artist_name   
0         Party       1000000                   75          0   AronChupa  \
1         Party       1000000                   75          1   AronChupa   
2         Party       1000000                   75          2       Lorde   
3         Party       1000000                   75          3       Lorde   
4         Party       1000000                   75          4       Lorde   

                              track_uri   
0  spotify:track:66U0ASk1VHZsqIkpMjKX3B  \
1  spotify:track:5MhsZlmKJG6X5kTHkdwC4B   
2  spotify:track:0GZoB8h0kqXn7XFm4Sj06k   
3  spotify:track:35kahykNu00FPysz3C2euR   
4  spotify:track:3G6hD9B2ZHOsgf4WfNu7X1   

                              artist_uri   
0  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh  \
1  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh   
2  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   
3  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   
4  spotify:artist:163tK9Wjr9P9DmM0AVK7lm   



In [24]:
df_spotify['playlist_pid'].nunique()

9000